In [1]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [11]:
data =  [(1,[2,3,4]), (2,[1,3]), (3,[4])] #simplified version : could be URL, LIST OF URLS

In [12]:
links = sc.parallelize(data).partitionBy(2).cache()

In [28]:
ranks = links.map(lambda x: (x[0], 1.0)) #init each rank to 1

In [29]:
#compute each url's contribution
def computeContribs(urls, rank):
    for url in urls:
        yield (url, rank / len(urls))

In [30]:
it_num = 10 #in practice it runs about 10 iterations
for i in range (it_num):
    contributions = links.join(ranks).flatMap(lambda x : computeContribs(x[1][0],x[1][1]))
    ranks = contributions.reduceByKey(lambda x,y : x+y).mapValues(lambda x : x*0.85+0.15)
ranks.collect()

[(1, 0.2430319557083253),
 (2, 0.21887340110327186),
 (3, 0.3119053568115972),
 (4, 0.4840575158297381)]

In [24]:
#Basically repeating this block 10+ times.
contributions = links.join(ranks).flatMap(lambda x : computeContribs(x[1][0],x[1][1]))
contributions.collect()
ranks = contributions.reduceByKey(lambda x,y : x+y).mapValues(lambda x : x*0.85+0.15)
ranks.collect()

[(1, 0.2430319557083253),
 (2, 0.21887340110327186),
 (3, 0.3119053568115972),
 (4, 0.4840575158297381)]